# Approach 2: The Semantic Detective 🕵️‍♀️

# **Section 1: Preprocessing**
+ In this section, we deal with all preprocessing steps required for the rest of this notebook, including importing libraries, installing necessary packages, initializing client module for Google BigQuery etc.
+ Our main tools for this project are `pandas` and `bigquery` from `google.cloud`.
+ Google Cloud's `bigframes` library is uninstalled because of version crashing in Kaggle's default environment.

In [2]:
# Install google-cloud-bigquery-storage for running BigQuery SQL without error
# Add -q to suppress verbose for the sake of readability 
!pip uninstall -q -y bigframes
!pip install -q google-cloud-bigquery-storage

In [3]:
# Import all libraries required for this project
import pandas as pd

from google.cloud import bigquery
from datetime import datetime, timedelta

## Define project and dataset ids
+ To create a database client for BigQuery, a project id is needed: `analog-delight-470708-d0`.
+ We also define dataset and table ids that have been imported from Google Cloud Buckets (GCB) to BigQuery. Please refer to our blog for details of the selected datasets.
    + `steam_game_list` contains the inventory of games available on Steam platform. There are some textual features such as `short description` and `tags` assigned to each game.
    + `review_data` contains players' reviews for some games on Steam.
+ The embedding model `llm_steam` is based on `text-embedding-004`. No further fine-tuning has been performed.

In [4]:
# Initialize BigQuery client with Google Cloud's project id
project_id = 'analog-delight-470708-d0'
client = bigquery.Client(project=project_id)

# We also define dataset and table ids
dataset_id = 'steam'
game_list_data = 'steam_game_list'
review_data = 'steam_reviews'

# We also define the name of text embedding models
embedding_model_name = 'llm_steam'

## Create primary keys for datasets in BigQuery


+ This helper function takes table and column names as arguments to check for the existence of a column in the schema of a table.
+ It is required to avoid repeating costly operations on BigQuery.

In [5]:
# Check whether a column exist in the table schema
def check_column_exists(dataset_id, table_id, name):
    # Given dataset_id and table_id, we retrieve its schema
    table_ref = client.dataset(dataset_id).table(table_id)
    table_schema = client.get_table(table_ref).schema

    # Loop through each field in the schema to determine whether a column exists.
    for field in table_schema:
        if field.name == name:
            return True
    return False

+ We convert *App ID* in `steam.steam_game_list` from string to integer as a new column called *app_id*.
+ This step facilitates table joining with steam's review data `steam.steam_reviews` in BigQuery.

In [6]:
# Check whether the primary key for game_list_data exists
game_list_data_pk = 'app_id'
exist_app_id = check_column_exists(dataset_id, game_list_data, game_list_data_pk)
print('Does the primary key exist? ' + str(exist_app_id))

# If it does not exist, generate it
if not exist_app_id:
    query = f"""
    alter table {project_id}.{dataset_id}.{game_list_data}
    add column if not exists {game_list_data_pk} integer;
    
    update {project_id}.{dataset_id}.{game_list_data}
    set {game_list_data_pk} = cast(`App ID` as integer)
    where true;
    
    alter table {project_id}.{dataset_id}.{game_list_data}
    add primary key ({game_list_data_pk}) not enforced;
    """
    result_pk = client.query(query)
    print(result_pk.result())

Does the primary key exist? True


# **Section 2: Generate Embeddings and Create Vector Indices**

## Generate embeddings for vector search
+ Two embedding vectors are created for the table `steam.steam_game_list` to improve search accuracy. They are based on two textual columns: `short description` and `tags`.
    + `short description` contains the description of the game e.g. "Plunger Knight is a platformer game about a lonely knight who lived in the Middle Age and has lost almost everything he had but had to face a new challenge: robots. In the series of his adventures, he travels the world and time to battle against them."
    + `tags` contains the tags assigned to each game on Steam e.g. `Action: 22, Adventure: 20, Casual: 20, Indie: 20, RPG: 19, Platformer: 12, Funny: 11, Puzzle-Platformer: 10`.

In [7]:
# This function generate embeddings using AI model if not exists
def create_embeddings(embeddings_name, embeddings_model_name, table_name, column_name):
    query = f"""
    alter table `{project_id}.{dataset_id}.{table_name}`
    add column if not exists {embeddings_name} array<float64>;

    update `{project_id}.{dataset_id}.{table_name}` as t
    set t.{embeddings_name} = e.ml_generate_embedding_result
    from (
        select distinct
            ml_generate_embedding_result,
            content
        from ml.generate_embedding(
            model `{project_id}.{dataset_id}.{embeddings_model_name}`,
            (select ifnull({column_name}, ' ') as content
              from `{project_id}.{dataset_id}.{table_name}`
            )
        )
    ) e
    where ifnull(t.{column_name}, ' ') = e.content
    """
    return client.query(query)

The embeddings columns `desc_embeddings` and `tags_embeddings` will be generated if they do not exist in the schema of `steam.steam_game_list` using the above function `create_embeddings()`.
+ `desc_embeddings` is extracted from the textual column `short description`.
+ `tags` is extracted from the textual column `tags`.

In [8]:
# Create text embeddings for 'short description' of each game available on steam
exist_desc = check_column_exists(dataset_id, game_list_data, "desc_embeddings")
print('Does the embeddings for short description of game exist? ' + str(exist_desc))
if not exist_desc:
    result_desc = create_embeddings("desc_embeddings", embedding_model_name, game_list_data, "`Short Description`")
    print(result_desc.result())

# Create text embeddings for 'tags' fof each game available on steam 
exist_tags = check_column_exists(dataset_id, game_list_data, "tags_embeddings")
print('Does the embeddings for tags of game exist? ' + str(exist_tags))
if not exist_tags:
    result_tags = create_embeddings("tags_embeddings", embedding_model_name, game_list_data, "tags")
    print(result_tags.result())

Does the embeddings for short description of game exist? True
Does the embeddings for tags of game exist? True


## Create Vector Index for High-performance Searching
Since the size of review data `steam.steam_reviews` is more than 8 GB, a vector index should be created to speed up the query process for embedding. 
There are in total 3 SQL statments which we will explain as follows:
+ Statement 1 (`create or replace table...`): The review data contains a textual column `review` which is used to create embeddings with AI model.

In [9]:
def create_table_reviews_embeddings(out_table_name, in_table_name):
    query = f"""
    create or replace table `{project_id}.{dataset_id}.{out_table_name}` as
    select *
    from ml.generate_embeddings(
    model `{project_id}.{dataset_id}.{embedding_model_name}`,
    (select Review as content, app_id from `{project_id}.{dataset_id}.{in_table_name}`),
    struct(
        true AS flatten_json_output, 
        'RETRIEVAL_DOCUMENT' as task_type
        )
    );

    create or replace table {dataset_id}.{out_table_name}_notnull as 
    select * from {dataset_id}.{out_table_name} 
    where ARRAY_LENGTH(ml_generate_embedding_result) > 0;

    create vector index if not exists embedding_index
    on {dataset_id}.{out_table_name}_notnull(ml_generate_embedding_result)
    storing(app_id)
    options(distance_type='COSINE', index_type='IVF');
    """
    return client.query(query)

In [10]:
table_review_embeddings = 'review_embeddings'
try:
    client.dataset(dataset_id).table(table_review_embeddings)
    print(f'Table {table_review_embeddings} already exists.')
except: 
    print(f'Table {table_review_embeddings} does not exist. A new table will be created.')
    handler = create_table_reviews_embeddings('review_embeddings', review_data)

Table review_embeddings already exists.


# **Section 3: Usecases of Google BigQuery AI in Product Positioning**

## Usecase 3.1 - Search a list of similar Steam games given a user query on game characteristics

The function `get_list_of_games` wraps the SQL query for retrieving games which are close to the user query in semantic. 
For example, a user query can be "`I would like to find a multi-person strategic game on farming in an open-world setting.`". 
To enhance accuracy, this SQL can select games with both description and tags close to the user query via an inner join on `app_id`.

*Parameters to tune:*
+ `number_of_games`: This parameter controls `top_k` in vector search on both embeddings. For the result to be accurate, the same semantic meaning should appear on both description and tags fields.
+ `min_reviews`: This parameter controls the popularity of games by the number of reviews they received, regardless of positive or negative.

*Expected results:*
A dataframe should be returned from `get_list_of_games` containing all columns in `steam.steam_game_list` for which the semantic meaning is close to the user query.

In [11]:
def get_list_of_games(user_input, number_of_games, min_reviews):
    embeddings = ["desc_embeddings", "tags_embeddings"]
    query = f"""
    select a.base.*
    from vector_search(
        (select {embeddings[0]}, name, app_id, `short description`, tags, `positive reviews`, `negative reviews` 
        from `{project_id}.{dataset_id}.{game_list_data}`
        where (`positive reviews` > {min_reviews}) or (`negative reviews` > {min_reviews})),
        '{embeddings[0]}',
        (select ml_generate_embedding_result, content as query 
        from ml.generate_embedding(
        model `{project_id}.{dataset_id}.{embedding_model_name}`,
            (select '{user_input}' as content))
        ),
        top_k => {number_of_games},
        distance_type => 'COSINE') as a
    inner join 
    vector_search(
        (select {embeddings[1]}, name, app_id, `short description`, tags, `positive reviews`, `negative reviews` 
        from `{project_id}.{dataset_id}.{game_list_data}`
        where (`positive reviews` > {min_reviews}) or (`negative reviews` > {min_reviews})),
        '{embeddings[1]}',
        (select ml_generate_embedding_result, content as query 
        from ml.generate_embedding(
        model `{project_id}.{dataset_id}.{embedding_model_name}`,
            (select '{user_input}' as content))
        ),
        top_k => {number_of_games},
        distance_type => 'COSINE') as b
        on a.base.app_id = b.base.app_id
    """
    df = client.query(query).to_dataframe()
    return df

In [12]:
number_of_games = 100
min_reviews = 1000

user_input = "I would like to find a multi-person strategic game on farming in an open-world setting."
df_retrieve = get_list_of_games(user_input, number_of_games, min_reviews)

In [13]:
# Convert pd.Series to python list object for postprocessing 
app_ids = df_retrieve['app_id'].values.tolist()
print('The list of relevant games (app_ids) retrieved by the user query: ', app_ids)

The list of relevant games (app_ids) retrieved by the user query:  [220260, 673950, 405710, 1248130, 1432860, 534370, 678900, 704030, 787860, 1123830, 495560, 840010]


In [14]:
# Calculate the metric column to measure good rating and sort the dataset in descending order.
df_retrieve['odd'] = df_retrieve['positive reviews']/df_retrieve['negative reviews']
df_sort = df_retrieve.iloc[:, 1:].sort_values('odd', ascending=False)
df_sort

,name,app_id,short description,tags,positive reviews,negative reviews,odd
1,Farm Together,673950,"Grow your own farm all by yourself, or coopera...","Agriculture: 362, Multiplayer: 338, Simulation...",17160,1070,16.037383
8,Farming Simulator 19,787860,The best-selling franchise takes a giant leap ...,"Simulation: 834, Farming Sim: 738, Multiplayer...",60886,3847,15.826878
0,Farming Simulator 2013 Titanium Edition,220260,"Animal husbandry, crops, sales… It's up to you...","Simulation: 513, Farming Sim: 509, Multiplayer...",4529,396,11.436869
11,Garden Paws,840010,You have inherited your grandparents farm as t...,"Exploration: 184, Sandbox: 178, Agriculture: 1...",1970,184,10.706522
4,Sun Haven,1432860,Build your farm and relationships with townsfo...,"Early Access: 362, Farming Sim: 329, Pixel Gra...",2672,251,10.645418
3,Farming Simulator 22,1248130,Create your farm and let the good times grow! ...,"Simulation: 644, Co-op: 628, Farming Sim: 625,...",33215,3797,8.747696
2,Staxel,405710,"Grow your farm, meet the villagers, and join y...","Farming Sim: 264, Cute: 248, Character Customi...",3451,860,4.012791
9,Farm Manager 2021,1123830,Get ready for a logistic challenge in the new ...,"Simulation: 79, Strategy: 66, Indie: 55, Agric...",1257,335,3.752239
6,Farmer's Dynasty,678900,Live – Build – Farm: Enjoy a unique mix of far...,"Simulation: 200, Farming Sim: 194, Life Sim: 1...",2424,796,3.045226
5,Pure Farming 2018,534370,Use the latest technology and state-of-the-art...,"Farming Sim: 164, Simulation: 155, Open World:...",1007,382,2.636126


## Usecase 3.2 - Search relevant reviews for the list of games given a query on product features

In [16]:
number_of_reviews = 20
user_input_age = 'Is this game easy to play for elderly?'
user_input_music = 'Tell me the music and visual aspects of this game.'
user_input_price = 'Is the price expensive or cheap or just affordable given the quality?'

def search_reviews(user_input, number_of_reviews):
    query = f"""
    select a.base.*
    from vector_search(
        (select ml_generate_embedding_result, app_id, content
        from `{project_id}.{dataset_id}.{table_review_embeddings}_notnull`
        where app_id in ({','.join([str(app_id) for app_id in app_ids])})),
        'ml_generate_embedding_result',
        (select ml_generate_embedding_result, content as query 
        from ml.generate_embedding(
        model `{project_id}.{dataset_id}.{embedding_model_name}`,
            (select '{user_input}' as content))
        ),
        top_k => {number_of_reviews},
        distance_type => 'COSINE') a
    """
    df = client.query(query).to_dataframe()
    return df

In [ ]:
df_age = search_reviews(user_input_age, number_of_reviews)
df_music = search_reviews(user_input_music, number_of_reviews)
df_price = search_reviews(user_input_price, number_of_reviews)

In [ ]:
def merge_dataset(df):
    return df.iloc[:, 1:].\
        merge(df_retrieve[['app_id', 'name', 'odd']], on='app_id').\
        sort_values('name')

In [ ]:
pd.set_option('display.max_colwidth', None)
print(f'Analysis: {user_input_age}')
merge_dataset(df_age)

In [ ]:
print(f'Analysis: {user_input_music}')
merge_dataset(df_music)

In [ ]:
print(f'Analysis: {user_input_price}')
merge_dataset(df_price)